In [ ]:
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt

import pandas as pd
import numpy as np
import math
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib inline


## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)


In [ ]:
class EMAAnalyzer(bt.Analyzer):
    
    def __init__(self):
        
        self.maperiod = self.strategy.params.maperiod
        
    def start(self):
        # Not needed ... but could be used
        self.init_cash = self.strategy.broker.cash 
        self.ntrade = 0

    def next(self):
        # Not needed ... but could be used
        pass

    def notify_trade(self, trade):
        self.ntrade += 1
    
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        
        outp = {
            'params': (self.maperiod),
            'profit': self.final_val - self.init_cash,
            'ntrade': self.ntrade
        }
        
        return outp


## Strategy Plot

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()
###################
#I'm going to add my 80% insample selection here:
OutofSample = math.floor(len(df) *.2)

in_sample_data = bt.feeds.PandasData(dataname = df.iloc[0:(len(df) - OutofSample)])
out_sample_data = bt.feeds.PandasData(dataname = df.iloc[(len(df) - OutofSample):])

# this is VS. the orig code:
##data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])
###################
# Add the Data Feed to Cerebro
cerebro.adddata(in_sample_data)
#I don't need the analyzer here because we are not optimizing any parameters
#cerebro.addanalyzer(EMAAnalyzer, _name='ema')

# Set our desired cash start
cerebro.broker.setcash(100000.0)

cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# this is the slippage parameters that are needed
cerebro.broker.set_slippage_fixed(.01)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
#cerebro.run()
thestrats = cerebro.run()
thestrat = thestrats[0]

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
#cerebro.plot(volume=False, iplot=False)
print('Sharpe Ratio:', thestrat.analyzers.mysharpe.get_analysis()['sharperatio'])


In [ ]:
cerebro.plot(volume=False)

## In-sample Optimization

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    maperiod=range(10, 31))

# Add the Data Feed to Cerebro
cerebro.adddata(in_sample_data)
cerebro.addanalyzer(EMAAnalyzer, _name='ema')


# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Set the commission
cerebro.broker.setcommission(commission=0.0)

# Run over everything
results = cerebro.run(maxcpus=4, optreturn=False)


In [ ]:
params  = []
n_trades = []
profit   = []
for res in results:
    r = res[0].analyzers.ema.get_analysis()
    params.append(r['params'])
    n_trades.append(r['ntrade'])
    profit.append(r['profit'])

In [ ]:
fig, (prof, nt) = plt.subplots(1,2, figsize=(15,5))
_=prof.hist(profit, bins=np.arange(6500,20000,1000))
_=nt.hist(n_trades, bins=np.arange(300,500,10))
prof.set_title('Profit Distribution')
nt.set_title('# of Trades Distribution')

## Out-of-Sample

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()
# Add a strategy
max_profit = profit.index(max(profit))
#This is my best performing parameter
best_maperiod = params[max_profit]
cerebro.addstrategy(TestStrategy, maperiod=best_maperiod)
#data = bt.feeds.PandasData(dataname = df['2018-01-01':'2018-06-01'])
# Add the Data Feed to Cerebro
cerebro.adddata(out_sample_data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run over everything
thestrats = cerebro.run()
thestrat = thestrats[0]
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Sharpe Ratio:', thestrat.analyzers.mysharpe.get_analysis()['sharperatio'] )
#cerebro.plot(volume=False, iplot=False)

In [ ]:
cerebro.plot(volume=False)

In [ ]:
cerebro.plot()